### Codes are inspired from: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [2]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.001
training_steps = 3000
batch_size = 512
display_step = 100

# Network parameters.
n_hidden_1 = 64 # 1st layer number of neurons.
n_hidden_2 = 128 # 2nd layer number of neurons.

In [3]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [6]:
# Creating model using keras Model

class NeuralNetwork(Model):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # First Hidden layer with 64 neuron
        self.hn1 = layers.Dense(n_hidden_1, activation = tf.nn.sigmoid)
        
        # Second Hidden layer with 64 neuron
        self.hn2 = layers.Dense(n_hidden_2, activation = tf.nn.sigmoid)
        
        # Third fully-connecter hidden layer.
        self.out = layers.Dense(num_classes)
        
    # Forward pass
    def call(self, x, is_training = False):
        x = self.hn1(x)
        x = self.hn2(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x
        
        

In [7]:
neuralNetwork = NeuralNetwork()

In [8]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [17]:
# Training process. 
def train(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neuralNetwork(x)
        loss = cross_entropy(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neuralNetwork.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [18]:

# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    train(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neuralNetwork(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 602.020569, accuracy: 0.746094
step: 200, loss: 255.733505, accuracy: 0.882812
step: 300, loss: 216.186371, accuracy: 0.886719
step: 400, loss: 175.950729, accuracy: 0.900391
step: 500, loss: 139.368729, accuracy: 0.935547
step: 600, loss: 161.316742, accuracy: 0.917969
step: 700, loss: 153.265747, accuracy: 0.925781
step: 800, loss: 126.786400, accuracy: 0.927734
step: 900, loss: 125.623543, accuracy: 0.925781
step: 1000, loss: 134.131210, accuracy: 0.933594
step: 1100, loss: 109.286461, accuracy: 0.949219
step: 1200, loss: 110.793625, accuracy: 0.929688
step: 1300, loss: 104.553780, accuracy: 0.947266
step: 1400, loss: 97.226860, accuracy: 0.943359
step: 1500, loss: 94.492851, accuracy: 0.953125
step: 1600, loss: 98.179367, accuracy: 0.947266
step: 1700, loss: 76.674934, accuracy: 0.957031
step: 1800, loss: 87.897301, accuracy: 0.941406
step: 1900, loss: 59.795399, accuracy: 0.974609
step: 2000, loss: 87.907318, accuracy: 0.957031
step: 2100, loss: 64.775421, accurac